In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats

In [16]:
wide_info_all = pd.read_csv('/home/mam_jupyter/jupyter_dir/artefact/leads_scoring_model/Python/dev/feature_all.csv', index_col = 0)

In [19]:
wide_info_all.shape

(1979997, 148)

In [17]:
wide_info_others = wide_info_all[wide_info_all[list(wide_info_all.columns[[i[0:2] == 'h_' for i in wide_info_all.columns]])].isna().T.all() == True]
wide_info_others = wide_info_others[['mobile', 'd_deal_flag'] + list(pd.read_csv('remained.csv')['action_origin'])] #drop columns with no lift difference

In [18]:
wide_info_others.shape

(1763560, 62)

In [45]:
#wide_info_others.to_csv('/home/mam_jupyter/jupyter_dir/artefact/leads_scoring_model/Python/dev/wide_info_others.csv')
wide_info_others.shape

(1763560, 62)

### Get Extra Information

In [23]:
lm_others = pd.read_csv('lift_matrix.csv', delimiter = ',')

In [33]:
city_lift = lm_others[lm_others['action_origin'] == 'c_city'][['action','1']]
city_lift['city'] = city_lift['action'].str.split('_', expand = True)[2]

### Data Cleaning

##### Detect and exclude outliers

In [ ]:
#wide_info_others[(np.abs(stats.zscore(wide_info_others)) < 3).all(axis=1)]

### Feature Transformation

##### Binary & Categorical Features

In [ ]:
wide_info_others.replace({'d_cust_type':{60261001:1, 60261002:2},
                        'c_sex':{'性别为男性':1, '性别为女性':2},
                        'c_age':{'20岁以下':1, '21-25岁':3, '26-30岁':3, '31-35岁':3, '36-40岁':3, 
                                 '41-45岁':3, '46-50岁':2, '51-55岁':2, '56-60岁':2, '60岁以上':4},
                        'c_province':{'贵州省':1, '云南省':1, '四川省':1, '重庆市':1, '福建省':2, '海南省':2, '广西壮族自治区':2, '安徽省':2,
                                     '江西省':2, '广东省':2, '上海市':2, '西藏自治区':2, '湖北省':2, '江苏省': 2},
                        'c_city_level':{'一线城市':3, '新一线城市':2, '二线城市':3, '三线城市':3, '四线城市':1, '五线城市':1},
                        'd_is_deposit_order':{0:2}})

In [ ]:
# deal with province
wide_info_others['c_province'] = [0 if i is np.nan else 3 if isinstance(i, int) == False for i in wide_info_others['c_province']]

# deal with city
city_1 = list(city_lift[city_lift['1'] <= 1]['city'])
city_2 = list(city_lift[(city_lift['1'] > 1) & (city_lift['1'] < 1.5)]['city'])
city_3 = list(city_lift[city_lift['1'] >= 1.5]['city'])
wide_info_others['c_city'] = [1 if i in city_1 else 2 if i in city_2 else 3 if i in city_3 else 0 for i in wide_info_others['c_city']]

In [ ]:
# low cov rate features but want to keep because they show great difference between w/o certain features
binary_features = ['d_fir_sec_visit_diff','d_avg_fir_sec_visit_diff','d_avg_firleads_firvisit_diff','d_trail_count_d30',
                        'd_visit_count_d15','d_firfollow_dealf_diff', 'd_firleads_firvisit_diff', 'd_leads_dtbt_ppt','c_register_time', 
                        'c_last_reach_platform_MG服务号','c_last_reach_platform_MGAPP','c_last_reach_platform_MG官网']

# binary column
wide_info_others[binary_features] = wide_info_others[binary_features].where(wide_info_others[binary_features].isnull(),1).fillna(0).astype(int)

In [71]:
wide_info_others['d_is_deposit_order'].isna().sum()

397251

In [72]:
wide_info_others['d_is_deposit_order'].value_counts()

0.0    1307354
1.0      58955
Name: d_is_deposit_order, dtype: int64

##### Binned Features

##### Deal with na

In [ ]:
# fillna with 0
wide_info_others[['d_cust_type','c_sex','c_age','c_city_level', 'c_province','d_is_deposit_order']].fillna(0)

#
cate_col = ['d_cust_type','c_sex','c_age','c_city','c_province','c_MG6','c_vertical_media','c_city_level','c_MGZS', 'd_is_deposit_order']